### ライブラリ読み込み & 設定 & クライアント初期化

In [ ]:

import os
from dotenv import load_dotenv
from azure.identity import DefaultAzureCredential
from azure.ai.projects import AIProjectClient
from azure.ai.projects.models import (
    PromptAgentDefinition,
    BingGroundingAgentTool,
    BingGroundingSearchToolParameters,
    BingGroundingSearchConfiguration,
)

# .env 読み込み
load_dotenv()

# Azure AI Project クライアント
project_client = AIProjectClient(
    endpoint=os.environ["project_endpoint"],
    credential=DefaultAzureCredential(),
)

# OpenAI クライアント
openai_client = project_client.get_openai_client()


### Bing Web Search Agent を作成

In [2]:
with project_client:
    agent = project_client.agents.create_version(
        agent_name="web-search-agent-vs",
        definition=PromptAgentDefinition(
            model="gpt-4.1",
            instructions="You are a helpful assistant that uses Bing Search.",
            tools=[
                BingGroundingAgentTool(
                    bing_grounding=BingGroundingSearchToolParameters(
                        search_configurations=[
                            BingGroundingSearchConfiguration(
                                project_connection_id=os.environ["BING_PROJECT_CONNECTION_ID"]
                            )
                        ]
                    )
                )
            ],
        ),
        description="Agent that uses Bing Search for grounding.",
    )

print(f"✅ エージェント作成完了 (名前： {agent.name}, バージョン: {agent.version})")

✅ エージェント作成完了 (名前： web-search-agent-vs, バージョン: 6)


In [5]:
# ====== ここを自由に変えて試せるエリア ======
user_prompt = "今日の東京の天気は？"
# ============================================

stream_response = openai_client.responses.create(
    stream=True,
    tool_choice="required",  # ツール呼び出しを必須にする
    input=user_prompt,
    extra_body={"agent": {"name": agent.name, "type": "agent_reference"}},
)

# ストリーミングでイベントを受け取りながら出力
for event in stream_response:
    if event.type == "response.created":
        print(f"レスポンスが生成されました（ID: {event.response.id}）")

    elif event.type == "response.output_text.delta":
        # トークンが流れてきたらそのまま表示
        print(event.delta, end="")

    elif event.type == "response.text.done":
        print("\n--- テキスト出力完了 ---")

    elif event.type == "response.output_item.done":
        if event.item.type == "message":
            item = event.item
            if item.content[-1].type == "output_text":
                text_content = item.content[-1]
                # URL 引用（ツールの情報源）があれば表示
                for annotation in text_content.annotations:
                    if annotation.type == "url_citation":
                        print(f"URL引用: {annotation.url}")

    elif event.type == "response.completed":
        print("\n--- 全ストリーム完了 ---")
        print("最終レスポンス:\n", event.response.output_text)


レスポンスが生成されました（ID: resp_0069dc4a2615f39600692dbbacadb88194aa36a7da7db8c63a）
本日の東京の天気は「晴れ時々曇り」です。気温は最高21℃、最低8℃前後で、12月としてはかなり暖かい一日になる見込みです。降水確率は午前中はほぼ0％、午後も10％程度と低く雨の心配はほとんどありません。風は北から南へ変わり、午後はやや強めに吹くことがあります。朝晩と日中で気温差が大きいので、脱ぎ着しやすい服装がおすすめです。URL引用: https://weather.yahoo.co.jp/weather/jp/13/4410.html
URL引用: https://tenki.jp/forecast/3/16/

--- 全ストリーム完了 ---
最終レスポンス:
 本日の東京の天気は「晴れ時々曇り」です。気温は最高21℃、最低8℃前後で、12月としてはかなり暖かい一日になる見込みです。降水確率は午前中はほぼ0％、午後も10％程度と低く雨の心配はほとんどありません。風は北から南へ変わり、午後はやや強めに吹くことがあります。朝晩と日中で気温差が大きいので、脱ぎ着しやすい服装がおすすめです。
